In [ ]:
import numpy as np
import skimage.io as io
from skimage.transform import resize
import random
import os
from functools import partial
import progressbar
from multiprocessing import Pool
from IPython.core.display import Image, display
from haar_features import *
from intagral_image import *
from Adaboost import *

LOADING_BAR_LENGTH = 50


In [ ]:
pos_training_path = 'trainingdata/faces'
neg_training_path = 'trainingdata/nonfaces'
pos_testing_path = 'trainingdata/faces/test'
neg_testing_path = 'trainingdata/nonfaces/test'


In [ ]:
print('Loading faces for training..')
faces_training = load_images(pos_training_path, trainingmax=400)
faces_ii_training = list(map(integral_image, faces_training))
print('..done. ' + str(len(faces_training)) + ' faces loaded.\n\nLoading non faces..')
non_faces_training = load_images(neg_training_path, trainingmax=600)
non_faces_ii_training = list(map(integral_image, non_faces_training))
print('..done. ' + str(len(non_faces_training)) + ' non faces loaded.\n')

print('Loading faces for testing..')
faces_testing = load_images(pos_testing_path,testmax=60,istest=True)
faces_ii_testing = list(map(integral_image, faces_testing))
print('..done. ' + str(len(faces_testing)) + ' faces loaded.\n\nLoading test non faces..')
non_faces_testing = load_images(neg_testing_path,testmax=60, istest=True)
non_faces_ii_testing = list(map(integral_image, non_faces_testing))
print('..done. ' + str(len(non_faces_testing)) + ' non faces loaded.\n')

In [ ]:
num_classifiers = 100
min_feature_height = 6
max_feature_height = 14
min_feature_width = 6
max_feature_width = 14

In [ ]:
#get best haar features classifers
classifiers = learn(faces_ii_training, non_faces_ii_training, num_classifiers, min_feature_width, max_feature_width, min_feature_height, max_feature_height)

In [ ]:
print('Testing selected classifiers..')
correct_faces = 0
correct_non_faces = 0
correct_faces = sum(ensemble_vote_all(faces_ii_testing, classifiers))
correct_non_faces = len(non_faces_testing) - sum(ensemble_vote_all(non_faces_ii_testing, classifiers))

print('..done.\n\nResult:\n      Faces: ' + str(correct_faces) + '/' + str(len(faces_testing))
      + '  (' + str((float(correct_faces) / len(faces_testing)) * 100) + '%)\n  non-Faces: '
      + str(correct_non_faces) + '/' + str(len(non_faces_testing)) + '  ('
      + str((float(correct_non_faces) / len(non_faces_testing)) * 100) + '%)')

In [ ]:
file = open('classifiersData3.txt', 'w')
for c in classifiers:
  file.write(str(c.type)[17:]+'\n')
  file.write(str(c.topLeft[0])+'\n')
  file.write(str(c.topLeft[1])+'\n')
  file.write(str(c.width)+'\n')
  file.write(str(c.height)+'\n')
  file.write(str(c.threshold)+'\n')
  file.write(str(c.parity)+'\n')
file.close()  